In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Nov 12 13:27:38 2017

@author: Georg
"""

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import backend as K
K.set_image_dim_ordering('th')
import os
os.chdir("/home/ubuntu/data/")


Using TensorFlow backend.


In [2]:
# path to the model weights files.
weights_path = '../vgg16_weights_tf_dim_ordering_tf_kernels.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'train'
validation_data_dir = 'validation'
test_dir = 'test'
nb_train_samples = 800
nb_validation_samples = 370
epochs = 20
batch_size = 5

input_tensor = Input(shape=(3,img_width,img_height))
base_model = applications.VGG16(weights='imagenet',include_top= False,input_tensor=input_tensor)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
top_model.load_weights(top_model_weights_path)
model = Model(inputs= base_model.input, outputs= top_model(base_model.output))

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:15]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        rotation_range=180,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

/usr/local/lib/python3.5/dist-packages/keras/applications/vgg16.py:182: UserWarning: You are using the TensorFlow backend, yet you are using the Theano image data format convention (`image_data_format="channels_first"`). For best performance, set `image_data_format="channels_last"` in your Keras config at ~/.keras/keras.json.
  warnings.warn('You are using the TensorFlow backend, yet you '


Found 841 images belonging to 2 classes.
Found 373 images belonging to 2 classes.


In [ ]:
# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., validation_steps=370, steps_per_epoch=160, epochs=20)`
  import sys


Epoch 1/20
160/160 [==============================] - 76s - loss: 0.4922 - acc: 0.8188 - val_loss: 0.6222 - val_acc: 0.6406
Epoch 2/20
160/160 [==============================] - 73s - loss: 0.4460 - acc: 0.8375 - val_loss: 0.8891 - val_acc: 0.6277
Epoch 3/20
160/160 [==============================] - 72s - loss: 0.4745 - acc: 0.8200 - val_loss: 0.7706 - val_acc: 0.6272
Epoch 4/20
160/160 [==============================] - 72s - loss: 0.4324 - acc: 0.8487 - val_loss: 0.6963 - val_acc: 0.6261
Epoch 5/20
160/160 [==============================] - 72s - loss: 0.4529 - acc: 0.8175 - val_loss: 0.8366 - val_acc: 0.6386
Epoch 6/20
160/160 [==============================] - 72s - loss: 0.4294 - acc: 0.8450 - val_loss: 0.6213 - val_acc: 0.6723
Epoch 7/20
160/160 [==============================] - 72s - loss: 0.4394 - acc: 0.8250 - val_loss: 0.6693 - val_acc: 0.6310
Epoch 8/20
160/160 [==============================] - 72s - loss: 0.4261 - acc: 0.8350 - val_loss: 0.6622 - val_acc: 0.6837
Epoch 9/